# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Nous avons effectué une analyse TWAS en utilisant à la fois S-PrediXcan (modèle Elastic Net ajusté pour la polygénicité) et FUSION. Nous avons volontairement inclus les modèles GTEx Pancreas pour les deux logiciels afin de permettre des comparaisons.

Les fichiers de résultats sont disponibles dans les points de contrôle et nous supposerons qu'ils sont :

In [ ]:
library(ggplot2)

Nous chargerons les résultats des deux analyses TWAS dans R et les fusionnerons.

In [ ]:
df_predixcan <- read.csv("results/twas_en_Pancreas.csv")

# Nous conservons les statistiques TWAS non calibrées dans un DF séparé.
df_predixcan_uncal <- df_predixcan[, c("gene", "gene_name", "uncalibrated_zscore", "uncalibrated_pvalue", "pred_perf_r2")]
names(df_predixcan_uncal) <- c("ENSG", "gene_label", "Z_PREDIXCAN_NO_CALIB", "P_PREDIXCAN_NO_CALIB", "R2_PREDIXCAN")

df_predixcan <- df_predixcan[, c("gene", "gene_name", "zscore", "pvalue", "pred_perf_r2")]
names(df_predixcan) <- c("ENSG", "gene_label", "Z_PREDIXCAN", "P_PREDIXCAN", "R2_PREDIXCAN")
nrow(df_predixcan)

In [ ]:
df_fusion <- read.csv(
    "results/fusion_GTEx_pancreas.tsv",
    sep="\t",
    stringsAsFactors=F
)
# Gardons seulement le modèle "enet" qui est la même approche de régression pénalisée
# que pour S-PrediXcan afin de maximiser la comparabilité.
df_fusion <- df_fusion[df_fusion$MODEL == "enet", ]
df_fusion <- df_fusion[, c("ID", "TWAS.Z", "TWAS.P", "MODELCV.R2")]
df_fusion <- df_fusion[complete.cases(df_fusion), ]
names(df_fusion) <- c("ENSG", "Z_FUSION", "P_FUSION", "R2_FUSION")
df_fusion$P_FUSION <- as.numeric(df_fusion$P_FUSION)
df_fusion$Z_FUSION <- as.numeric(df_fusion$Z_FUSION)

nrow(df_fusion)

In [ ]:
merged_uncalibrated <- merge(df_predixcan_uncal, df_fusion, by="ENSG")
merged <- merge(df_predixcan, df_fusion, by="ENSG")
nrow(merged)

Comparons d'abord les valeurs P pour les modèles FUSION et S-PrediXcan avant la calibration pour la polygénicité. Nous afficherons le seuil de P <= 0,05 / 1058, le nombre total de gènes qui étaient disponibles dans les résultats pour les deux algorithmes.

In [ ]:
p_thresh <- -log10(0.05 / 1058)

ggplot(merged_uncalibrated, aes(x = -log10(P_FUSION), y = -log10(P_PREDIXCAN_NO_CALIB))) +
    geom_rect(aes(xmin = -Inf, xmax = p_thresh, ymin = p_thresh, ymax = Inf), 
              fill = "#eff5ff", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = p_thresh, ymax = Inf), 
              fill = "#ffeff5", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = -Inf, ymax = p_thresh), 
              fill = "#f5ffef", inherit.aes = FALSE) +
    geom_point() +
    geom_vline(xintercept = -log10(0.05 / 1058), linetype = "dotted", color = "#888888", size = 1) +
    geom_hline(yintercept = -log10(0.05 / 1058), linetype = "dotted", color = "#888888", size = 1) +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red") +
    theme_bw()

<div class="alert alert-info">
    <p>
        Nous avons identifié trois régions dans le graphique en utilisant des couleurs différentes. Comment pouvez-vous interpréter les points dans chacune de ces régions ? Quelles conclusions pouvez-vous tirer sur les différentes méthodes TWAS ?
    </p>
    <p>
        En utilisant les données disponibles, pouvez-vous identifier un gène qui est significatif en utilisant les deux approches ? Seulement FUSION ? Seulement S-PrediXcan ?
    </p>
</div>

In [ ]:
# Vous pouvez utiliser cette cellule pour explorer les données !

Maintenant, nous répéterons le graphique mais en utilisant les statistiques de S-PrediXcan qui ont été calibrées pour la polygénicité. Utilisez le graphique ci-dessous pour discuter :

<div class="alert alert-info">
    Comment l'ajustement pour la polygénicité influence-t-il les valeurs P ? À quelle approche devrions-nous faire confiance ?
</div>

In [ ]:
p_thresh <- -log10(0.05 / 1058)

ggplot(merged, aes(x = -log10(P_FUSION), y = -log10(P_PREDIXCAN))) +
    geom_rect(aes(xmin = -Inf, xmax = p_thresh, ymin = p_thresh, ymax = Inf), 
              fill = "#eff5ff", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = p_thresh, ymax = Inf), 
              fill = "#ffeff5", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = -Inf, ymax = p_thresh), 
              fill = "#f5ffef", inherit.aes = FALSE) +
    geom_point() +
    geom_vline(xintercept = p_thresh, linetype = "dotted", color = "#888888", size = 1) +
    geom_hline(yintercept = p_thresh, linetype = "dotted", color = "#888888", size = 1) +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red") +
    theme_bw()